In [2]:
from elasticsearch import Elasticsearch

In [7]:
# Connect to Elasticsearch
es = Elasticsearch(
    ["http://localhost:9200"],
    verify_certs=False,  # Set to True if your Elasticsearch has a valid SSL certificate
)

In [10]:
# Index name and document
index_name = "first_index"

# Specify transport options using Elasticsearch.options()
transport_options = {
    'ignore': 400
}

# Create the index with transport options
es.indices.create(index=index_name, params=transport_options)

/tmp/ipykernel_7487/2271307005.py:10: DeprecationWarning: The 'params' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index_name, params=transport_options)
/tmp/ipykernel_7487/2271307005.py:10: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, params=transport_options)


ObjectApiResponse({'error': {'root_cause': [{'type': 'resource_already_exists_exception', 'reason': 'index [first_index/KmXY6nG0SjymrWZBl0K6lw] already exists', 'index_uuid': 'KmXY6nG0SjymrWZBl0K6lw', 'index': 'first_index'}], 'type': 'resource_already_exists_exception', 'reason': 'index [first_index/KmXY6nG0SjymrWZBl0K6lw] already exists', 'index_uuid': 'KmXY6nG0SjymrWZBl0K6lw', 'index': 'first_index'}, 'status': 400})

In [ ]:

es.indices.exists(index=index_name)

HeadApiResponse(False)

In [ ]:

es.indices.exists(index=index_name)

HeadApiResponse(False)

In [ ]:

es.indices.exists(index=index_name)

HeadApiResponse(False)

In [ ]:

es.indices.exists(index=index_name)

HeadApiResponse(False)

In [11]:
es.indices.delete(index=index_name)

ObjectApiResponse({'acknowledged': True})

In [12]:

es.indices.exists(index=index_name)

HeadApiResponse(False)

In [20]:
# Insert and Get Query
doc1 = {"city":"Srinagar", "Country": "India"}
doc2 = {"city":"Moscow", "Country": "Russia"}
es.index(index="cities", id=1, body=doc1)

ObjectApiResponse({'_index': 'cities', '_id': '1', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1})

In [17]:
es.index(index="cities", id=2, body=doc2)


ObjectApiResponse({'_index': 'cities', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

In [18]:
res = es.get(index="cities", id=1)

In [19]:
res

ObjectApiResponse({'_index': 'cities', '_id': '1', '_version': 2, '_seq_no': 1, '_primary_term': 1, 'found': True, '_source': {'city': 'Srinagar', 'Country': 'India'}})

In [21]:
res["_source"]

{'city': 'Srinagar', 'Country': 'India'}

In [22]:
# Different serach query for matching docs
doc3 = {"sentence":"Today is a SUNNY day"}
doc4 = {"sentence":"Today is a BRIGHT-SUNNY day"}

In [23]:
es.index(index="english", id=1, body=doc3)

ObjectApiResponse({'_index': 'english', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [24]:
es.index(index="english", id=2, body=doc4)


ObjectApiResponse({'_index': 'english', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

In [25]:
res1 = es.search(index="english", body={
    "from":0,
    "size":0,
    "query":{
        "match":{
            "sentence":"SUNNY"
        }
    }
})

res1

ObjectApiResponse({'took': 387, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

es.search: This method is used to perform a search operation on Elasticsearch. It takes parameters such as the index name, the search query body, and other optional parameters.

index="english": Specifies the index on which the search operation should be performed. In this case, it is the "english" index.

body: This is the query body that defines the search request. It's a JSON object that specifies the search criteria.

"from": 0: Specifies the starting index from where the search results should be retrieved. In this case, it is set to 0, indicating that the results should start from the first match.

"size": 0: Specifies the number of hits (search results) to be returned. Setting it to 0 indicates that only the aggregations and metadata should be returned, but no actual search hits.

"query": This is the key indicating the start of the query definition.

"match": This is a type of query that performs a full-text search on the specified field. In this case, it is searching for documents where the "sentence" field matches the term "SUNNY."

In [26]:

"""   
    res1 using a "match" query looks for documents containing individual tokens from the search term.
    res2 using a "match_phrase" query looks for documents where the entire search term appears as a phrase in the specified field.
"""
    
res2 = es.search(index="english", body={
    "from":0,
    "size":0,
    "query":{
        "match_phrase":{
            "sentence":"SUNNY"
        }
    }
})

res2


ObjectApiResponse({'took': 11, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [27]:
res3 = es.search(index="english", body={
    "from":0,
    "size":0,
    "query":{
        "match_phrase":{
            "sentence":"bright-SUNNY"
        }
    }
})

res3

ObjectApiResponse({'took': 46, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [28]:
"""
    The "term" query is used for exact matching of terms in Elasticsearch.
    It looks for documents where the exact term specified in the query matches the term in the specified field.
    Unlike the "match" and "match_phrase" queries, 
    the "term" query doesn't analyze the search term; it looks for the exact term in the inverted index.
"""
res4 = es.search(index="english", body={
    "from":0,
    "size":0,
    "query":{
        "term":{
            "sentence":"bright-SUNNY"
        }
    }
})

res4

ObjectApiResponse({'took': 16, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [29]:
# Combining queries ------> must, must_not, should

"""
"bool" Query:

    The "bool" query is a compound query that allows you to combine multiple queries using boolean logic.
    It supports various clauses, including:
        "must":
            Documents must match this query. It's similar to the logical AND operation.
        "must_not":
            Documents must not match this query. It's similar to the logical NOT operation.
        "should":
            This represents optional matching. 
            Documents that match this query will contribute to the final score but are not required to match.
"""
res5 = es.search(index="english", body={
    "from":0,
    "size":0,
    "query":{
        "bool":{
            "must_not":{
                "match":{
                    "sentence":"bright"
                }
            }, 
            "should":{
                "match":{
                    "sentence":"sunny"
                }
            }
        }
    }
})

res5


ObjectApiResponse({'took': 34, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [30]:
# Combining queries ------> must, must_not, should
res6 = es.search(index="english", body={
    "from":0,
    "size":1,
    "query":{
        "bool":{
            "must_not":{
                "match":{
                    "sentence":"bright"
                }
            }, 
            "should":{
                "match":{
                    "sentence":"sunny"
                }
            }
        }
    }
})

res6


ObjectApiResponse({'took': 156, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.18936403, 'hits': [{'_index': 'english', '_id': '1', '_score': 0.18936403, '_source': {'sentence': 'Today is a SUNNY day'}}]}})

In [33]:
# Regular Expressions in ES

# Add another document
doc5 = {"sentence":"Today is a rainy day"}
es.index(index="english", id=3, body=doc5)



ObjectApiResponse({'_index': 'english', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

In [34]:
es.get(index="english", id=1)
es.get(index="english", id=2)
es.get(index="english", id=3)

ObjectApiResponse({'_index': 'english', '_id': '3', '_version': 1, '_seq_no': 2, '_primary_term': 1, 'found': True, '_source': {'sentence': 'Today is a rainy day'}})

In [35]:
"""
    "regexp" Query:

    The "regexp" query allows you to search for documents based on a regular expression pattern.
    In this case, the regular expression pattern is ".*" which essentially matches any string (zero or more characters). 
    This means it will match any document where the "sentence" field contains at least one character.
"""

res7 = es.search(index="english", body={
    "from":0,
    "size":3,
    "query":{
        "regexp":{
            "sentence":".*"
        }
    }
})

res7

ObjectApiResponse({'took': 63, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'english', '_id': '1', '_score': 1.0, '_source': {'sentence': 'Today is a SUNNY day'}}, {'_index': 'english', '_id': '2', '_score': 1.0, '_source': {'sentence': 'Today is a BRIGHT-SUNNY day'}}, {'_index': 'english', '_id': '3', '_score': 1.0, '_source': {'sentence': 'Today is a rainy day'}}]}})